Set up dependencies

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os.path
pd.set_option('precision', 2)

### Put all matches into a single dataframe

In [2]:
matches = pd.read_csv('all_t20i_05-16.csv')
match_date = matches.set_index('match_id').date.to_dict()
files = sorted(match_date.items(), key=lambda value: value[1])

In [3]:
def get_req_rr(row):
    runs_to_get = row['target'] - row['cumul_runs']
    balls_remaining = 120 - row['cumul_balls']
    if balls_remaining > 0 and runs_to_get >= 0:
        return 6.*runs_to_get/balls_remaining
    else:
        return None

In [4]:
all_data = pd.DataFrame()
for idx, fname in enumerate(files):
    if os.path.exists('cleaned_data/%d_summary.csv'%fname[0]):
        ind_game = pd.read_csv('cleaned_data/%d_summary.csv'%fname[0])#.sort_values('inning').sort_values('ovr')
        ind_game['date'] = fname[1]
        t1_score = ind_game[ind_game.inning == 1].runs_w_extras.sum()
        ind_game['target'] = ind_game.inning.apply(lambda x: t1_score if x == 2 else None)
        ind_game['current_run_rate'] = ind_game.apply(lambda row: 6.*row['cumul_runs']/row['cumul_balls'], axis=1)
        ind_game['required_run_rate'] = ind_game.apply(lambda row: get_req_rr(row) if row['inning'] == 2 else None, axis=1)
        ind_game['ovr_range'] = ind_game.ovr.apply(lambda x: 'first_6' if x < 6 else 'middle_9' if x < 15 else 'last_5')
        all_data = all_data.append(ind_game)
    if idx%50 == 0:
        print idx, len(files)
all_data['balls'] = 1
all_data = all_data.reset_index(drop=True)
cols = all_data.columns.tolist()
all_data = all_data[cols[1:]]
all_data['year'] = all_data['date'].apply(lambda x: x.split('-')[0])

0 508
50 508
100 508
150 508
200 508
250 508
300 508
350 508
400 508
450 508
500 508


In [5]:
batsman_team_dict = all_data.drop_duplicates(subset = ['batsman_name']).set_index('batsman_name')['batting_team'].to_dict()
bowler_team_dict = all_data.drop_duplicates(subset = ['bowler_name']).set_index('bowler_name')['bowling_team'].to_dict()

In [6]:
major_teams = ['Australia', 'New Zealand', 'England', 'South Africa', 'West Indies', 'Sri Lanka',
               'Pakistan', 'India', 'Bangladesh']

### Getting all innings of a team

In [7]:
team_innings = pd.DataFrame()
team_innings['opposition'] = all_data.groupby(['batting_team', 'date'])['bowling_team'].apply(lambda x: x.tolist()[0])
team_innings['inning'] = all_data.groupby(['batting_team', 'date'])['inning'].apply(lambda x: x.tolist()[0])
team_innings['opposition_score'] = all_data.groupby(['bowling_team', 'date'])['runs_w_extras'].agg('sum')
team_innings['runs_scored'] = all_data.groupby(['batting_team', 'date'])['runs_w_extras'].agg('sum')
team_innings['win'] = team_innings['runs_scored'] > team_innings['opposition_score']
team_innings['num_out'] = all_data.groupby(['batting_team', 'date'])['wicket'].agg('sum')
team_innings['balls_faced'] = all_data.groupby(['batting_team', 'date'])['ovr'].nunique()
team_innings['num_fours'] = all_data.groupby(['batting_team', 'date'])['runs_batter'].apply(lambda x: sum([i == 4 for i in x.tolist()]))
team_innings['num_sixes'] = all_data.groupby(['batting_team', 'date'])['runs_batter'].apply(lambda x: sum([i == 6 for i in x.tolist()]))
team_innings['num_dots'] = all_data.groupby(['batting_team', 'date'])['runs_batter'].apply(lambda x: sum([i == 0 for i in x.tolist()]))
team_innings = team_innings.reset_index()
team_innings['year'] = team_innings['date'].apply(lambda x: int(x.split('-')[0]))
team_innings['year'] = team_innings['year'].apply(lambda x: '05-07' if x <= 2007 else '08-10' if x <= 2010 else '11-13' if x <=2013 else '14-16')
team_innings['run_rate'] = 6.*team_innings.apply(lambda row: row['runs_scored']/max(row['balls_faced'], 1), axis=1)

### Aggregating team batting performance by year

In [8]:
team_year = team_innings.groupby(['batting_team', 'year'])[
    ['win', 'runs_scored', 'balls_faced', 'num_fours', 'num_sixes','num_dots']].agg('sum')\
    .join(team_innings.groupby(['batting_team', 'year'])['date'].count()).reset_index()
team_year.columns = team_year.columns.tolist()[:-1] + ['num_games']
team_year = team_year[['batting_team', 'year', 'num_games', 'win', 'runs_scored', 'balls_faced',
                            'num_fours', 'num_sixes', 'num_dots']]
team_year['win_percentage'] = 1.*team_year['win']/team_year['num_games']
team_year['prop_runs_sixes'] = 1.*(6.*team_year['num_sixes'])/team_year['runs_scored']
team_year['prop_runs_boundaries'] = 1.*(6.*team_year['num_sixes'] + 4.*team_year['num_fours'])/team_year['runs_scored']
team_year['run_rate'] = 6.*team_year['runs_scored']/team_year['balls_faced']
team_year['dot_prop'] = 1.*team_year['num_dots']/team_year['balls_faced']
team_year['balls_per_six'] = 1.*team_year['balls_faced']/team_year['num_sixes']
team_year['balls_per_four'] = 1.*team_year['balls_faced']/team_year['num_fours']
team_year['balls_per_boundary'] = 1.*team_year['balls_faced']/(team_year['num_fours'] + team_year['num_sixes'])

In [9]:
team_year[team_year.batting_team == 'India']

,batting_team,year,num_games,win,runs_scored,balls_faced,num_fours,num_sixes,num_dots,win_percentage,prop_runs_sixes,prop_runs_boundaries,run_rate,dot_prop,balls_per_six,balls_per_four,balls_per_boundary
19,India,05-07,8,6,1331,948,122,46,416,0.75,0.21,0.57,8.42,0.44,20.61,7.77,5.64
20,India,08-10,18,8,2681,2022,195,103,888,0.44,0.23,0.52,7.96,0.44,19.63,10.37,6.79
21,India,11-13,19,11,2891,2170,257,73,834,0.58,0.15,0.51,7.99,0.38,29.73,8.44,6.58
22,India,14-16,21,15,3076,2345,272,83,976,0.71,0.16,0.52,7.87,0.42,28.25,8.62,6.61


### Analyze team by batting position

In [10]:
team_batting_pos = pd.DataFrame()
team_batting_pos['runs_scored'] = all_data.groupby(['batting_team', 'batting_order'])['runs_w_extras'].agg('sum')
team_batting_pos['num_out'] = all_data.groupby(['batting_team', 'batting_order'])['wicket'].agg('sum')
team_batting_pos['balls_faced'] = all_data.groupby(['batting_team', 'batting_order'])['ovr'].count()
team_batting_pos['num_fours'] = all_data.groupby(['batting_team', 'batting_order'])['runs_batter'].apply(lambda x: sum([i == 4 for i in x.tolist()]))
team_batting_pos['num_sixes'] = all_data.groupby(['batting_team', 'batting_order'])['runs_batter'].apply(lambda x: sum([i == 6 for i in x.tolist()]))
team_batting_pos['num_dots'] = all_data.groupby(['batting_team', 'batting_order'])['runs_batter'].apply(lambda x: sum([i == 0 for i in x.tolist()]))
team_batting_pos = team_batting_pos.reset_index()
team_batting_pos['strike_rate'] = 100.*team_batting_pos.apply(lambda row: row['runs_scored']/max(row['balls_faced'], 1), axis=1)
team_batting_pos['average'] = 1.*team_batting_pos.apply(lambda row: row['runs_scored']/max(row['num_out'], 1), axis=1)

### Getting all innings of a batsman

In [11]:
batsman_innings = pd.DataFrame()
batsman_innings['inning'] = all_data.groupby(['batsman_name', 'date'])['inning'].apply(lambda x: x.tolist()[0])
batsman_innings['opposition'] = all_data.groupby(['batsman_name', 'date'])['bowling_team'].apply(lambda x: x.tolist()[0])
batsman_innings['runs_scored'] = all_data.groupby(['batsman_name', 'date'])['runs_batter'].agg('sum')
batsman_innings['num_out'] = all_data.groupby(['batsman_name', 'date'])['wicket'].agg('sum')
batsman_innings['balls_faced'] = all_data.groupby(['batsman_name', 'date'])['ovr'].count()
batsman_innings['num_fours'] = all_data.groupby(['batsman_name', 'date'])['runs_batter'].apply(lambda x: sum([i == 4 for i in x.tolist()]))
batsman_innings['num_sixes'] = all_data.groupby(['batsman_name', 'date'])['runs_batter'].apply(lambda x: sum([i == 6 for i in x.tolist()]))
batsman_innings['num_dots'] = all_data.groupby(['batsman_name', 'date'])['runs_batter'].apply(lambda x: sum([i == 0 for i in x.tolist()]))
batsman_innings['batting_order'] = all_data.groupby(['batsman_name', 'date'])['batting_order'].apply(lambda x: x.dropna().values[0])
batsman_innings['wicket_method'] = all_data.groupby(['batsman_name', 'date'])['wicket_method'].apply(lambda x: x.dropna().tolist())
batsman_innings['wicket_method'] = batsman_innings['wicket_method'].apply(lambda x: 'not out' if len(x) == 0 else x[-1])
batsman_innings['wicket_method'] = batsman_innings.apply(lambda row: 'not out' if row['num_out'] == 0 else row['wicket_method'], axis=1)
for i in ['caught', 'not out', 'leg before wicket', 'bowled', 'hit wicket',
       'run out', 'stumped', 'retired not out (hurt)']:
    batsman_innings[i] = batsman_innings['wicket_method'] == i
batsman_innings = batsman_innings.reset_index()
batsman_innings['team'] = batsman_innings['batsman_name'].apply(batsman_team_dict.get)
batsman_innings['year'] = batsman_innings['date'].apply(lambda x: int(x.split('-')[0]))
batsman_innings['year'] = batsman_innings['year'].apply(lambda x: '05-07' if x <= 2007 else '08-10' if x <= 2010 else '11-13' if x <=2013 else '14-16')
batsman_innings['strike_rate'] = 100.*batsman_innings.apply(lambda row: row['runs_scored']/max(row['balls_faced'], 1), axis=1)

### Aggregating batsman performances by batting order

In [12]:
batsman_params = ['runs_scored', 'balls_faced', 'num_fours', 'num_sixes', 'num_dots', 'num_out',
                  'caught', 'not out', 'leg before wicket', 'bowled', 'hit wicket',
                  'run out', 'stumped', 'retired not out (hurt)']
batsman_order = batsman_innings.groupby(['batsman_name', 'batting_order'])[batsman_params].agg('sum')\
    .join(batsman_innings.groupby(['batsman_name', 'batting_order'])['date'].nunique())
batsman_order.columns = batsman_order.columns.tolist()[:-1] + ['num_innings']
batsman_order = batsman_order.reset_index()
for i in ['caught', 'not out', 'leg before wicket', 'bowled', 'hit wicket',
       'run out', 'stumped', 'retired not out (hurt)']:
    batsman_order[i] = 1.*batsman_order[i]/batsman_order['num_innings']
batsman_order['team'] = batsman_order['batsman_name'].apply(batsman_team_dict.get)
batsman_order['average'] = batsman_order.apply(lambda row: row['runs_scored']/max(row['num_out'], 1), axis=1)
batsman_order['runs_per_inning'] = batsman_order['runs_scored']/batsman_order['num_innings']
batsman_order['strike_rate'] = 100.*batsman_order.apply(lambda row: row['runs_scored']/max(row['balls_faced'], 1), axis=1)
batsman_order['impact'] = batsman_order['strike_rate']*batsman_order['runs_per_inning']
batsman_order['prop_dot'] = 100.*batsman_order['num_dots']/batsman_order['balls_faced']
batsman_order['balls_per_six'] = batsman_order['balls_faced']/batsman_order['num_sixes']
batsman_order['balls_per_boundary'] = batsman_order['balls_faced']/(batsman_order['num_sixes'] + batsman_order['num_fours'])
batsman_order['prop_run_boundary'] = 100.*(6.*batsman_order['num_sixes'] + 4.*batsman_order['num_fours'])/batsman_order['runs_scored']
batsman_order['prop_run_six'] = 100.*(6.*batsman_order['num_sixes'])/batsman_order['runs_scored']

### Aggregating batsman performances by year

In [13]:
batsman_params = ['runs_scored', 'balls_faced', 'num_fours', 'num_sixes', 'num_dots', 'num_out',
                  'caught', 'not out', 'leg before wicket', 'bowled', 'hit wicket',
                  'run out', 'stumped', 'retired not out (hurt)']
batsman_year = batsman_innings.groupby(['batsman_name', 'year'])[batsman_params].agg('sum')\
    .join(batsman_innings.groupby(['batsman_name', 'year'])['date'].nunique())
batsman_year.columns = batsman_year.columns.tolist()[:-1] + ['num_innings']
batsman_year = batsman_year.reset_index()
for i in ['caught', 'not out', 'leg before wicket', 'bowled', 'hit wicket',
       'run out', 'stumped', 'retired not out (hurt)']:
    batsman_year[i] = 1.*batsman_year[i]/batsman_year['num_innings']
batsman_year['team'] = batsman_year['batsman_name'].apply(batsman_team_dict.get)
batsman_year['average'] = batsman_year.apply(lambda row: row['runs_scored']/max(row['num_out'], 1), axis=1)
batsman_year['runs_per_inning'] = batsman_year['runs_scored']/batsman_year['num_innings']
batsman_year['strike_rate'] = 100.*batsman_year.apply(lambda row: row['runs_scored']/max(row['balls_faced'], 1), axis=1)
batsman_year['impact'] = batsman_year['strike_rate']*batsman_year['runs_per_inning']
batsman_year['prop_dot'] = 100.*batsman_year['num_dots']/batsman_year['balls_faced']
batsman_year['balls_per_six'] = batsman_year['balls_faced']/batsman_year['num_sixes']
batsman_year['balls_per_boundary'] = batsman_year['balls_faced']/(batsman_year['num_sixes'] + batsman_year['num_fours'])
batsman_year['prop_run_boundary'] = 100.*(6.*batsman_year['num_sixes'] + 4.*batsman_year['num_fours'])/batsman_year['runs_scored']
batsman_year['prop_run_six'] = 100.*(6.*batsman_year['num_sixes'])/batsman_year['runs_scored']

In [14]:
batsman_year[batsman_year.batsman_name.str.contains('Yuvraj')]

,batsman_name,year,runs_scored,balls_faced,num_fours,num_sixes,num_dots,num_out,caught,not out,...,team,average,runs_per_inning,strike_rate,impact,prop_dot,balls_per_six,balls_per_boundary,prop_run_boundary,prop_run_six
1454,Yuvraj Singh,05-07,179,103,10,15,36,4,0.67,0.33,...,India,44.75,29.83,173.79,5184.63,34.95,6.87,4.12,72.63,50.28
1455,Yuvraj Singh,08-10,376,278,23,23,110,14,0.60,0.20,...,India,26.86,25.07,135.25,3390.31,39.57,12.09,6.04,61.17,36.70
1456,Yuvraj Singh,11-13,313,219,20,21,83,9,0.55,0.27,...,India,34.78,28.45,142.92,4066.79,37.90,10.43,5.34,65.81,40.26
1457,Yuvraj Singh,14-16,214,215,19,10,109,9,0.67,0.25,...,India,23.78,17.83,99.53,1775.04,50.70,21.50,7.41,63.55,28.04


In [15]:
params = ['batsman_name', 'team', 'year', 'runs_scored', 'balls_faced', 'num_innings', 'runs_per_inning',
      'strike_rate', 'impact', 'prop_dot', 'balls_per_six', 'prop_run_six']
batsman_year[(batsman_year.balls_faced >= 50) & (batsman_year.num_innings >= 5) & (batsman_year.team.isin(major_teams))]\
    .sort_values('impact', ascending=False)[params].head()

,batsman_name,team,year,runs_scored,balls_faced,num_innings,runs_per_inning,strike_rate,impact,prop_dot,balls_per_six,prop_run_six
6,Aaron Finch,Australia,11-13,366,211,9,40.67,173.46,7054.03,30.81,12.41,27.87
1417,Virat Kohli,India,14-16,740,570,15,49.33,129.82,6404.68,30.53,31.67,14.59
244,Chris Gayle,West Indies,05-07,193,126,5,38.60,153.17,5912.54,42.86,11.45,34.20
197,Brendon McCullum,New Zealand,11-13,781,536,20,39.05,145.71,5689.93,36.94,14.89,27.66
109,Andrew Symonds,Australia,05-07,337,203,10,33.70,166.01,5594.53,25.12,20.30,17.80


### Getting all performances of a bowler

In [16]:
bowler_spells = pd.DataFrame()
bowler_spells['inning'] = all_data.groupby(['bowler_name', 'date'])['inning'].apply(lambda x: x.tolist()[0])
bowler_spells['opposition'] = all_data.groupby(['bowler_name', 'date'])['batting_team'].apply(lambda x: x.tolist()[0])
bowler_spells['runs_conceded'] = all_data.groupby(['bowler_name', 'date'])['runs_batter'].agg('sum')
bowler_spells['wickets_taken'] = all_data.groupby(['bowler_name', 'date'])['wicket'].agg('sum')
bowler_spells['balls_bowled'] = all_data.groupby(['bowler_name', 'date'])['ovr'].count()
bowler_spells['num_dots'] = all_data.groupby(['bowler_name', 'date'])['runs_batter'].apply(lambda x: sum([i == 0 for i in x.tolist()]))
bowler_spells = bowler_spells.reset_index()
bowler_spells['team'] = bowler_spells['bowler_name'].apply(bowler_team_dict.get)
bowler_spells['year'] = bowler_spells['date'].apply(lambda x: int(x.split('-')[0]))
bowler_spells['year'] = bowler_spells['year'].apply(lambda x: '05-07' if x <= 2007 else '08-10' if x <= 2010 else '11-13' if x <=2013 else '14-16')
bowler_spells['economy_rate'] = 6.*bowler_spells.apply(lambda row: row['runs_conceded']/max(row['balls_bowled'], 1), axis=1)

### Aggregating bowling performances by year

In [17]:
bowler_params = ['runs_conceded', 'balls_bowled', 'wickets_taken', 'num_dots']
bowler_year = bowler_spells.groupby(['bowler_name', 'year'])[bowler_params].agg('sum')\
    .join(bowler_spells.groupby(['bowler_name', 'year'])['date'].nunique())
bowler_year.columns = bowler_year.columns.tolist()[:-1] + ['num_spells']
bowler_year = bowler_year.reset_index()
bowler_year['team'] = bowler_year['bowler_name'].apply(bowler_team_dict.get)
bowler_year['average'] = bowler_year['runs_conceded']/bowler_year['wickets_taken']
bowler_year['wickets_per_spell'] = bowler_year['wickets_taken']/bowler_year['num_spells']
bowler_year['strike_rate'] = bowler_year['balls_bowled']/bowler_year['wickets_taken']
bowler_year['economy_rate'] = 6.*bowler_year['runs_conceded']/bowler_year['balls_bowled']
bowler_year['prop_dot'] = 100.*bowler_year['num_dots']/bowler_year['balls_bowled']

In [18]:
bowler_year[bowler_year.bowler_name.str.contains('Ashwin')]

,bowler_name,year,runs_conceded,balls_bowled,wickets_taken,num_dots,num_spells,team,average,wickets_per_spell,strike_rate,economy_rate,prop_dot
759,Ravichandran Ashwin,08-10,68,50,2,16,2,India,34.00,1.00,25.00,8.16,32.00
760,Ravichandran Ashwin,11-13,483,405,13,162,17,India,37.15,0.76,31.15,7.16,40.00
761,Ravichandran Ashwin,14-16,387,429,36,209,18,India,10.75,2.00,11.92,5.41,48.72


In [19]:
bowler_year[(bowler_year.balls_bowled >= 100) & (bowler_year.team.isin(major_teams))].sort_values('economy_rate').head()

,bowler_name,year,runs_conceded,balls_bowled,wickets_taken,num_dots,num_spells,team,average,wickets_per_spell,strike_rate,economy_rate,prop_dot
201,Dale Steyn,11-13,179,234,17,139,11,South Africa,10.53,1.55,13.76,4.59,59.40
1011,Umar Gul,05-07,153,193,12,94,8,Pakistan,12.75,1.50,16.08,4.76,48.70
365,Imran Tahir,11-13,104,131,7,47,6,South Africa,14.86,1.17,18.71,4.76,35.88
832,Samuel Badree,14-16,233,284,20,156,12,West Indies,11.65,1.67,14.20,4.92,54.93
208,Daniel Vettori,08-10,391,469,22,208,20,New Zealand,17.77,1.10,21.32,5.00,44.35


Most effective - low economy rate and low strike rate

In [20]:
bowler_year[(bowler_year.balls_bowled >= 100) & (bowler_year.team.isin(major_teams))
           & (bowler_year.economy_rate <= 6.5) & (bowler_year.strike_rate <= 16) & (bowler_year.year =='14-16')][
    ['bowler_name', 'year', 'strike_rate', 'economy_rate', 'average']].sort_values('average', ascending=1).head()

,bowler_name,year,strike_rate,economy_rate,average
761,Ravichandran Ashwin,14-16,11.92,5.41,10.75
832,Samuel Badree,14-16,14.20,4.92,11.65
227,David Wiese,14-16,11.38,6.30,11.95
494,Krishmar Santokie,14-16,11.88,6.24,12.35
512,Lasith Malinga,14-16,12.52,6.15,12.84


Stop runs but don't take wickets

In [21]:
bowler_year[(bowler_year.balls_bowled >= 100) & (bowler_year.team.isin(major_teams))
           & (bowler_year.economy_rate <= 6.5) & (bowler_year.strike_rate > 16) & (bowler_year.year =='14-16')][
    ['bowler_name', 'year', 'strike_rate', 'economy_rate', 'average']].sort_values('economy_rate', ascending=1).head()

,bowler_name,year,strike_rate,economy_rate,average
943,Sunil Narine,14-16,20.59,5.07,17.41
652,Mustafizur Rahman,14-16,19.00,5.08,16.08
124,Bhuvneshwar Kumar,14-16,25.78,5.12,22.00
341,Hardik Pandya,14-16,18.10,5.50,16.60
86,Angelo Mathews,14-16,24.44,5.89,24.00


Wicket taking but expensive

In [22]:
bowler_year[(bowler_year.balls_bowled >= 100) & (bowler_year.team.isin(major_teams))
           & (bowler_year.economy_rate > 6.5) & (bowler_year.strike_rate <= 16) & (bowler_year.year =='14-16')][
    ['bowler_name', 'year', 'strike_rate', 'economy_rate', 'average']].sort_values('average', ascending=1).head()

,bowler_name,year,strike_rate,economy_rate,average
228,David Willey,14-16,8.67,8.31,12.00
49,Al-Amin Hossain,14-16,11.17,6.56,12.20
669,Nathan Coulter-Nile,14-16,12.81,6.64,14.19
202,Dale Steyn,14-16,12.20,7.18,14.60
122,Beuran Hendricks,14-16,13.38,7.96,17.75


Expensive and non-wicket taking

In [23]:
bowler_year[(bowler_year.balls_bowled >= 100) & (bowler_year.team.isin(major_teams))
           & (bowler_year.economy_rate > 6.5) & (bowler_year.strike_rate > 16) & (bowler_year.year =='14-16')][
    ['bowler_name', 'year', 'strike_rate', 'economy_rate', 'average']].sort_values('strike_rate', ascending=0).head()

,bowler_name,year,strike_rate,economy_rate,average
403,James Tredwell,14-16,52.50,7.14,62.5
413,Jean-Paul Duminy,14-16,36.60,7.64,46.6
983,Tim Bresnan,14-16,35.60,9.13,54.2
386,Jade Dernbach,14-16,28.14,10.02,47.0
631,Mohit Sharma,14-16,27.80,7.94,36.8


### Best teams for chasing under pressure

Chasing with the required rate above 9

In [24]:
chasing_under_pressure_team = all_data[all_data.required_run_rate > 9].groupby('batting_team')[['runs_w_extras', 'balls', 'wicket']].agg('sum')
chasing_under_pressure_team['run_rate'] = 6.*chasing_under_pressure_team['runs_w_extras']/chasing_under_pressure_team['balls']
chasing_under_pressure_team['average'] = 1.*chasing_under_pressure_team['runs_w_extras']/chasing_under_pressure_team['wicket']

In [25]:
chasing_under_pressure_team.sort_values('run_rate', ascending=False).head()

,runs_w_extras,balls,wicket,run_rate,average
batting_team,,,,,
India,1615,1190,68,8.14,23.75
England,3074,2301,140,8.02,21.96
Australia,2533,1964,147,7.74,17.23
South Africa,1940,1540,101,7.56,19.21
West Indies,2205,1792,115,7.38,19.17


### Best batsman for chasing under pressure

Strike rate while chasing with the required rate above 9

In [26]:
chasing_under_pressure = all_data[all_data.required_run_rate > 9].groupby('batsman_name')[['runs_batter', 'balls', 'wicket']].agg('sum')
chasing_under_pressure['strike_rate'] = 1.*chasing_under_pressure['runs_batter']/chasing_under_pressure['balls']
chasing_under_pressure['average'] = 1.*chasing_under_pressure['runs_batter']/chasing_under_pressure['wicket']

In [27]:
chasing_under_pressure[chasing_under_pressure.balls > 60].sort_values('average', ascending=False).head()

,runs_batter,balls,wicket,strike_rate,average
batsman_name,,,,,
MS Dhoni,220,181,3,1.22,73.33
Virat Kohli,122,93,2,1.31,61.00
Virender Sehwag,107,64,2,1.67,53.50
Matt Machan,106,76,2,1.39,53.00
Alex Hales,420,299,9,1.40,46.67


### Team performance by innings

In [28]:
first_inning_team = team_innings[team_innings.inning == 1].groupby('batting_team')[
    ['win', 'runs_scored', 'num_out', 'balls_faced', 'num_fours', 'num_sixes', 'num_dots']].agg('sum')\
    .join(team_innings[team_innings.inning == 1].groupby('batting_team')['date'].count())
first_inning_team.columns = first_inning_team.columns.tolist()[:-1] + ['total_matches']
first_inning_team['average'] = 1.*first_inning_team['runs_scored']/first_inning_team['num_out']
first_inning_team['run_rate'] = 6.*first_inning_team['runs_scored']/first_inning_team['balls_faced']
first_inning_team['balls_per_six'] = 1.*first_inning_team['balls_faced']/first_inning_team['num_sixes']
first_inning_team['balls_per_boundary'] = 1.*first_inning_team['balls_faced']/first_inning_team['num_fours']
first_inning_team['win_rate'] = 1.*first_inning_team['win']/first_inning_team['total_matches']

In [29]:
second_inning_team = team_innings[team_innings.inning == 2].groupby('batting_team')[
    ['win', 'runs_scored', 'num_out', 'balls_faced', 'num_fours', 'num_sixes', 'num_dots']].agg('sum')\
    .join(team_innings[team_innings.inning == 2].groupby('batting_team')['date'].count())
second_inning_team.columns = second_inning_team.columns.tolist()[:-1] + ['total_matches']
second_inning_team['average'] = 1.*second_inning_team['runs_scored']/second_inning_team['num_out']
second_inning_team['run_rate'] = 6.*second_inning_team['runs_scored']/second_inning_team['balls_faced']
second_inning_team['balls_per_six'] = 1.*second_inning_team['balls_faced']/second_inning_team['num_sixes']
second_inning_team['balls_per_boundary'] = 1.*second_inning_team['balls_faced']/second_inning_team['num_fours']
second_inning_team['win_rate'] = 1.*second_inning_team['win']/second_inning_team['total_matches']

In [30]:
first_inning_team.sort_values('win_rate', ascending=False).head()

,win,runs_scored,num_out,balls_faced,num_fours,num_sixes,num_dots,total_matches,average,run_rate,balls_per_six,balls_per_boundary,win_rate
batting_team,,,,,,,,,,,,,
Afghanistan,15,3371,147,2617,233,148,1137,22,22.93,7.73,17.68,11.23,0.68
Pakistan,34,8050,335,6335,653,233,2711,53,24.03,7.62,27.19,9.70,0.64
Sri Lanka,28,7084,260,5247,664,176,2200,44,27.25,8.10,29.81,7.90,0.64
South Africa,28,6904,253,5157,563,229,2060,45,27.29,8.03,22.52,9.16,0.62
England,23,6001,241,4378,489,211,1741,37,24.90,8.22,20.75,8.95,0.62


In [31]:
second_inning_team.sort_values('win_rate', ascending=False).head()

,win,runs_scored,num_out,balls_faced,num_fours,num_sixes,num_dots,total_matches,average,run_rate,balls_per_six,balls_per_boundary,win_rate
batting_team,,,,,,,,,,,,,
Netherlands,12,2433,114,1979,195,72,843,18,21.34,7.38,27.49,10.15,0.67
Papua New Guinea,2,312,19,264,20,14,139,3,16.42,7.09,18.86,13.20,0.67
India,21,4572,146,3445,398,135,1458,32,31.32,7.96,25.52,8.66,0.66
South Africa,24,5414,201,4139,464,168,1777,39,26.94,7.85,24.64,8.92,0.62
Afghanistan,11,2533,112,2022,218,75,912,18,22.62,7.52,26.96,9.28,0.61


### Batsman performance by innings

In [32]:
first_inning_bat = batsman_innings[batsman_innings.inning == 1].groupby('batsman_name')[
    ['runs_scored', 'num_out', 'balls_faced', 'num_fours', 'num_sixes', 'num_dots', 'caught', 'not out',
     'leg before wicket', 'bowled', 'hit wicket', 'run out', 'stumped', 'retired not out (hurt)']].agg('sum')
first_inning_bat['average'] = 1.*first_inning_bat['runs_scored']/first_inning_bat['num_out']
first_inning_bat['strike_rate'] = 100.*first_inning_bat['runs_scored']/first_inning_bat['balls_faced']
first_inning_bat['balls_per_six'] = 1.*first_inning_bat['balls_faced']/first_inning_bat['num_sixes']
first_inning_bat['balls_per_boundary'] = 1.*first_inning_bat['balls_faced']/first_inning_bat['num_fours']
for dismissal in [u'caught', u'not out', u'leg before wicket', u'bowled', u'hit wicket', u'run out', u'stumped',
       u'retired not out (hurt)']:
    first_inning_bat[dismissal] = 1.*first_inning_bat[dismissal]/first_inning_bat['num_out']

In [33]:
second_inning_bat = batsman_innings[batsman_innings.inning == 2].groupby('batsman_name')[
    ['runs_scored', 'num_out', 'balls_faced', 'num_fours', 'num_sixes', 'num_dots', 'caught', 'not out',
     'leg before wicket', 'bowled', 'hit wicket', 'run out', 'stumped', 'retired not out (hurt)']].agg('sum')
second_inning_bat['average'] = 1.*second_inning_bat['runs_scored']/second_inning_bat['num_out']
second_inning_bat['strike_rate'] = 100.*second_inning_bat['runs_scored']/second_inning_bat['balls_faced']
second_inning_bat['balls_per_six'] = 1.*second_inning_bat['balls_faced']/second_inning_bat['num_sixes']
second_inning_bat['balls_per_boundary'] = 1.*second_inning_bat['balls_faced']/second_inning_bat['num_fours']
for dismissal in [u'caught', u'not out', u'leg before wicket', u'bowled', u'hit wicket', u'run out', u'stumped',
       u'retired not out (hurt)']:
    second_inning_bat[dismissal] = 1.*second_inning_bat[dismissal]/second_inning_bat['num_out']

### Bowler performance by innings

### Aggregating batsman performances in the first innings and the second innings

In [34]:
first_inning_bat = all_data[all_data.inning==1].groupby('batsman_name')[['runs_batter', 'balls', 'wicket']].agg('sum')
second_inning_bat = all_data[all_data.inning==2].groupby('batsman_name')[['runs_batter', 'balls', 'wicket']].agg('sum')

In [35]:
first_inning_bat['average'] = 1.*first_inning_bat['runs_batter']/first_inning_bat['wicket']
first_inning_bat['strike_rate'] = 100.*first_inning_bat['runs_batter']/first_inning_bat['balls']

In [36]:
second_inning_bat['average'] = 1.*second_inning_bat['runs_batter']/second_inning_bat['wicket']
second_inning_bat['strike_rate'] = 100.*second_inning_bat['runs_batter']/second_inning_bat['balls']

In [37]:
first_inning_bat[first_inning_bat.balls > 100].sort_values('average', ascending=False).head()

,runs_batter,balls,wicket,average,strike_rate
batsman_name,,,,,
Matt Machan,224,190,3,74.67,117.89
Aaron Finch,502,288,8,62.75,174.31
Andrew Symonds,292,173,5,58.40,168.79
Ed Joyce,160,151,3,53.33,105.96
Devon Smith,159,133,3,53.00,119.55


In [38]:
second_inning_bat[second_inning_bat.balls > 100].sort_values('average', ascending=False).head()

,runs_batter,balls,wicket,average,strike_rate
batsman_name,,,,,
Matthew Hayden,266,183,2,133.00,145.36
Virat Kohli,717,564,9,79.67,127.13
David Obuya,153,151,2,76.50,101.32
Ashish Bagai,131,120,2,65.50,109.17
Richard Levi,176,115,3,58.67,153.04


### Aggregating batting performance in the first innings by over

In [39]:
first_inning_first_6 = all_data[(all_data.inning==1) & (all_data.ovr_range == 'first_6')].groupby('batsman_name')[['runs_batter', 'balls', 'wicket']].agg('sum')
first_inning_middle_9 = all_data[(all_data.inning==1) & (all_data.ovr_range == 'middle_9')].groupby('batsman_name')[['runs_batter', 'balls', 'wicket']].agg('sum')
first_inning_last_5 = all_data[(all_data.inning==1) & (all_data.ovr_range == 'last_5')].groupby('batsman_name')[['runs_batter', 'balls', 'wicket']].agg('sum')

In [40]:
for i in [first_inning_first_6, first_inning_middle_9, first_inning_last_5]:
    i['strike_rate'] = 100.*i['runs_batter']/i['balls']
    i['average'] = 1.*i['runs_batter']/i['wicket']

In [41]:
first_inning_first_6[(first_inning_first_6.balls > 60)].sort_values('strike_rate', ascending=False).head()

,runs_batter,balls,wicket,strike_rate,average
batsman_name,,,,,
Aaron Finch,235,138,3,170.29,78.33
Joe Root,100,62,1,161.29,100.00
Anton Devcich,101,67,1,150.75,101.00
Kevin Pietersen,246,168,2,146.43,123.00
Ryan Watson,89,63,4,141.27,22.25


In [42]:
first_inning_middle_9[(first_inning_middle_9.balls > 100)].sort_values('strike_rate', ascending=False).head()

,runs_batter,balls,wicket,strike_rate,average
batsman_name,,,,,
Shane Watson,275,161,7,170.81,39.29
Malcolm Waller,180,112,2,160.71,90.00
Chris Gayle,371,249,5,149.00,74.20
Brendon McCullum,496,336,13,147.62,38.15
Kevin Pietersen,333,227,12,146.70,27.75


In [43]:
team_innings.to_csv('summary_csvs/team_innings.csv', encoding='utf-8')
team_year.to_csv('summary_csvs/team_year.csv', encoding='utf-8')
team_batting_pos.to_csv('summary_csvs/team_batting_pos.csv', encoding='utf-8')
batsman_innings.to_csv('summary_csvs/batsman_innings.csv', encoding='utf-8')
batsman_order.to_csv('summary_csvs/batsman_order.csv', encoding='utf-8')
batsman_year.to_csv('summary_csvs/batsman_year.csv', encoding='utf-8')
chasing_under_pressure_team.to_csv('summary_csvs/chasing_under_pressure_team.csv', encoding='utf-8')
first_inning_team.to_csv('summary_csvs/first_inning_team.csv', encoding='utf-8')
second_inning_team.to_csv('summary_csvs/second_inning_team.csv', encoding='utf-8')
first_inning_bat.to_csv('summary_csvs/first_inning_bat.csv', encoding='utf-8')
second_inning_bat.to_csv('summary_csvs/second_inning_bat.csv', encoding='utf-8')
first_inning_first_6.to_csv('summary_csvs/first_inning_first_6.csv', encoding='utf-8')
first_inning_middle_9.to_csv('summary_csvs/first_inning_middle_9.csv', encoding='utf-8')
first_inning_last_5.to_csv('summary_csvs/first_inning_last_5.csv', encoding='utf-8')
bowler_spells.to_csv('summary_csvs/bowler_spells.csv', encoding='utf-8')
bowler_year.to_csv('summary_csvs/bowler_year.csv', encoding='utf-8')

In [45]:
team_innings.to_pickle('summary_pickles/team_innings.pkl')
team_year.to_pickle('summary_pickles/team_year.pkl')
team_batting_pos.to_pickle('summary_pickles/team_batting_pos.pkl')
batsman_innings.to_pickle('summary_pickles/batsman_innings.pkl')
batsman_order.to_pickle('summary_pickles/batsman_order.pkl')
batsman_year.to_pickle('summary_pickles/batsman_year.pkl')
chasing_under_pressure_team.to_pickle('summary_pickles/chasing_under_pressure_team.pkl')
first_inning_team.to_pickle('summary_pickles/first_inning_team.pkl')
second_inning_team.to_pickle('summary_pickles/second_inning_team.pkl')
first_inning_bat.to_pickle('summary_pickles/first_inning_bat.pkl')
second_inning_bat.to_pickle('summary_pickles/second_inning_bat.pkl')
first_inning_first_6.to_pickle('summary_pickles/first_inning_first_6.pkl')
first_inning_middle_9.to_pickle('summary_pickles/first_inning_middle_9.pkl')
first_inning_last_5.to_pickle('summary_pickles/first_inning_last_5.pkl')